<a href="https://colab.research.google.com/github/karaage0703/covid19_memo/blob/master/signate_covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新型コロナウイルスの感染状況を分析するためのノートブック

データを可視化するために、個人的に作成しているものです。

内容の正確性を保証するものではありませんので、ご注意下さい。


以下リポジトリでPR受付中です。

https://github.com/karaage0703/covid19_memo

## データのダウンロード
SIGNATEのCOVID-19チャレンジ（フェーズ1）で作成されたオープンデータ（Google Spread Sheet)を用いて分析

https://signate.jp/competitions/260

In [0]:
!wget --no-check-certificate --output-document=data.csv 'https://docs.google.com/spreadsheets/d/1CnQOf6eN18Kw5Q6ScE_9tFoyddk4FBwFZqZpt_tMOm4/export?gid=0&format=csv'

--2020-04-01 07:52:24--  https://docs.google.com/spreadsheets/d/1CnQOf6eN18Kw5Q6ScE_9tFoyddk4FBwFZqZpt_tMOm4/export?gid=0&format=csv
Resolving docs.google.com (docs.google.com)... 108.177.125.100, 108.177.125.102, 108.177.125.101, ...
Connecting to docs.google.com (docs.google.com)|108.177.125.100|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data.csv’

data.csv                [   <=>              ]   1.09M  1.96MB/s    in 0.6s    

2020-04-01 07:52:31 (1.96 MB/s) - ‘data.csv’ saved [1139676]



## 死亡率と性別の関係

SIGNATEのフォーラムの以下の分析例を、Google Colaboratoryで動作するように改変

https://signate.jp/competitions/261/discussions/20200401125031-59

データをpandasを使って読み込み、内容を確認する

In [0]:
import pandas as pd

# SIGNATE COVID-2019 Dataset からデータを読み込み、ステータスと性別を抽出
df = pd.read_csv("data.csv")
df = df[["ステータス","性別"]]

# ステータスのフラグを変換
df["ステータス"] = df["ステータス"].replace({0: '罹患中', 1: '回復', 2: '死亡'})

f = lambda x: '{:.2%}'.format(x)

ステータス別の人数と割合

In [0]:
status_cnt = df["ステータス"].value_counts()
status_ratio = df["ステータス"].value_counts(normalize=True).apply(f) 

status_table = pd.concat([status_cnt, status_ratio], axis=1)
status_table.columns= ["人数","割合(%)"]
status_table

,人数,割合(%)
罹患中,1786,91.54%
回復,140,7.18%
死亡,25,1.28%


性別の人数と割合

In [0]:
gender_cnt = df["性別"].value_counts()
gender_ratio = df["性別"].value_counts(normalize=True).apply(f)
 
gender_table = pd.concat([gender_cnt, gender_ratio],  axis=1)
gender_table.columns= ["人数","割合"]
gender_table

,人数,割合
男性,1111,58.23%
女性,792,41.51%
非公表,5,0.26%


性別による死亡率の違い

In [0]:
df["ステータス"] = df["ステータス"].replace({'罹患中': '生存', "回復": '生存', 2: '死亡'})
ct = pd.crosstab(df["性別"],df["ステータス"])
ct

ステータス,死亡,生存
性別,,
女性,6,786
男性,17,1094
非公表,1,4


In [0]:
print("死亡における男性の割合：{:.0%}".format(ct.loc["男性","死亡"]/ct.loc[:,"死亡"].sum()))

死亡における男性の割合：71%


p値の確認

In [0]:
# Fisher's exact test
import scipy.stats as st
_, p = st.fisher_exact(ct[0:2])
print(f'p値 = {p :.3f}')

p値 = 0.142


# References